In [ ]:
import cv2
import numpy as np

from glob import glob
from random import randint
from scipy.optimize import linear_sum_assignment

In [ ]:
paths = sorted(glob('./CS585-Cells/*'))

In [ ]:
def getCont(path):
    img = cv2.imread(path, 0)
    img = cv2.blur(img, (5, 5))
    
    a_thres_output = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 17, -5)
    contours, _ = cv2.findContours(a_thres_output, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    return contours

def getCent(c):
    points = [i[0] for i in c]
    x, y = zip(*points)
    x, y = np.mean(x), np.mean(y)
    return (x, y)

det = lambda x: [getCent(i) for i in getCont(x)]

In [ ]:
# dets = [det(i) for i in paths]
def readFile(fp):
    with open(fp, 'r') as file:
        data = file.read().strip()
    
    data = data.split('\n')
    f = lambda x: np.array(list(map(float, x.split(','))), np.float32)
    return list(map(f, data))



detp = sorted(glob('./cellAnns/*'))
dets = [readFile(i) for i in detp]


In [ ]:
class Obj:
    def __init__(self, start_measure):
        self.filter = cv2.KalmanFilter(4, 2)
        self.color = (randint(80, 255), randint(80, 255), randint(80, 255))

        
        self.filter.measurementMatrix = np.array([[1, 0, 0, 0],
                                                  [0, 1, 0, 0]], np.float32)

        self.filter.transitionMatrix = np.array([[1, 0, 1, 0],
                                                 [0, 1, 0, 1],
                                                 [0, 0, 1, 0],
                                                 [0, 0, 0, 1]], np.float32)
        #  process noise
        self.filter.processNoiseCov = 0.3 * np.array([[1, 0, 0, 0],
                                                 [0, 1, 0, 0],
                                                 [0, 0, 1, 0],
                                                 [0, 0, 0, 1]], np.float32)
        
        
        for i in range(10):
            self.filter.predict()
            self.filter.correct(start_measure)
        
        
        self.history = []
        self.tracks = 0
        
        self._miss = 0
        
    def getPred(self):
            pred =  self.filter.predict()
            pred = np.array([pred[0][0], pred[1][0]])

            self.history.append(pred)
        
            return pred
    
    def update(self, measure):
        self.filter.correct(measure)
        self._miss = 0
        self.tracks += 1
        
    def miss(self):
        self._miss += 1
        if self._miss >= 6:
            return True
        else:
            
            return False
        
    def draw(self, img):
        if self.tracks>=4:
            pts = np.array([[np.rint(i).astype('int32').tolist() for i in self.history[3:]]])
            cv2.polylines(img, pts, False, self.color, 1, lineType=cv2.LINE_8)
        return img    

In [ ]:
sq = lambda x: x*x
dist = lambda x, y: sum(sq(x-y))**0.5

def getCM(est, meas):
    x, y = len(est), len(meas)
    
    cm = 70*np.ones((x+y, x+y), dtype = np.float64)
    for i in range(x):
        for j in range(y):
            cm[i, x+j] = cm[x+j, i] = dist(est[i], meas[j])
    return cm

Have a folder in the same directory called cellResults

In [ ]:
objs =  [Obj(i) for i in dets[0]]

# fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
# video=cv2.VideoWriter('video.avi', fourcc, 1,(1024, 1024))

counter = 0
for ind, det in enumerate(dets[1:]):
    CM = getCM([obj.getPred() for obj in objs], det)
    obj_v = [False for i in objs]
    mea_v = [False for i in det]
    
    row_ind, col_ind = linear_sum_assignment(CM)
    
    prop = (row_ind!=col_ind)
    row_ind, col_ind = row_ind[prop], col_ind[prop]
    
    mi = (row_ind<len(objs)).sum()
    row_ind, col_ind = row_ind[:mi], col_ind[:mi]
    
    col_ind -= len(objs)
    prop = (col_ind>=0)
    row_ind, col_ind = row_ind[prop], col_ind[prop]
    
#     print(row_ind, len(objs))
#     print(col_ind, len(det))
    for i in range(row_ind.shape[0]):
        objs[row_ind[i]].update(det[col_ind[i]])
        obj_v[row_ind[i]], mea_v[col_ind[i]] = True, True
    
    miss = [i for i in [i for i, j in enumerate(obj_v) if not j] if objs[i].miss()]
    miss = [j-i for i, j in enumerate(miss)]
    for i in miss:
        objs.pop(i)
         
    
    for i in [i for i, j in enumerate(mea_v) if not j]:
        objs.append(Obj(det[i]))
    
    img = cv2.imread(paths[ind+1], 0)
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    
    for obj in objs:
        img = obj.draw(img)
    
    cv2.imwrite("./cellResults/"+str(counter)+".jpg",img)
    counter += 1
#     video.write(img)
    # cv2.waitKey(0);

# cv2.destroyAllWindows()
# video.release()

In [ ]:
import os 
from os.path import isfile, join
pathIn= 'cellResults/'
pathOut = 'cellResults/cells.mp4'
# It was a 50s video and I generated 150 frames
# Therefore I have 3fps
# I'll use 10 to make the video run faster
fps = 10
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
print(files)

['0.jpg', '1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg', '51.jpg', '52.jpg', '53.jpg', '54.jpg', '55.jpg', '56.jpg', '57.jpg', '58.jpg', '59.jpg', '60.jpg', '61.jpg', '62.jpg', '63.jpg', '64.jpg', '65.jpg', '66.jpg', '67.jpg', '68.jpg', '69.jpg', '70.jpg', '71.jpg', '72.jpg', '73.jpg', '74.jpg', '75.jpg', '76.jpg', '77.jpg', '78.jpg', '79.jpg', '80.jpg', '81.jpg', '82.jpg', '83.jpg', '84.jpg', '85.jpg', '86.jpg', '87.jpg', '88.jpg', '89.jpg', '90.jpg', '91.jpg', '92.jpg', '93.jpg', '94.jpg', '95.jpg', '96.jpg', '97.jpg', '98.jpg', '99.jpg', '100.jpg'

In [ ]:
for i in range(len(files)):
    filename=pathIn + files[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)

In [ ]:
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    print(i)
    out.write(frame_array[i])
out.release()